In [146]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import nltk
import pandas as pd
from ast import literal_eval
from collections import Counter
from scipy.optimize import fmin_l_bfgs_b
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import multiprocessing
import random
from tqdm import tqdm
import optimizeTopicVectors as ot
import time

In [3]:
# hoseong's class
from STMD import *
from ASUM import *
from ASUM_Embedding import *
from preprocess import *

In [4]:
# load data
# work_path = "/media/hs-ubuntu/data/dataset/MasterThesis/STMD_data/"
work_path = "E:/dataset/MasterThesis/STMD_data/"

data = pd.read_csv(work_path + "preprocess_complete_Electronics.csv")

In [126]:
def extract_sample(data, brand_name, count, ratio = 1, random_state = 42):
    brand = data[data['brand']==brand_name]
    pos_reviews = brand[brand.overall >= 4]
    neg_reviews = brand[brand.overall <= 2]
    if ratio == 1:
        pos_sample = pos_reviews.sample(count, random_state=random_state)
        neg_sample = neg_reviews.sample(count, random_state=random_state)
        df = pd.concat([pos_sample, neg_sample], axis=0)
    else:
        df = brand.sample(count * 2, random_state = random_state)
    df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
    return df

In [127]:
# brand = ['Apple', 'Samsung','Canon']
# brand_df = data[data.brand.isin(brand)]
# brand_df.reset_index(drop=True, inplace=True)
# #긍정, 부정 반반씩
# pos_reviews = brand_df[brand_df.overall >= 4]
# neg_reviews = brand_df[brand_df.overall <= 2]
# pos_sample = pos_reviews.sample(3500, random_state=23)
# neg_sample = neg_reviews.sample(3500, random_state=42)
# df = pd.concat([pos_sample, neg_sample], axis=0)
# df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
# df.reset_index(drop=True, inplace=True)

# apple, samsung, canon 1:1:1
apple = extract_sample(data, 'Apple', 500, ratio = 1)
samsung = extract_sample(data, 'Samsung', 500, ratio = 1)
canon = extract_sample(data, 'Canon', 500, ratio = 1)
df = pd.concat([apple, samsung, canon], axis=0)
df.reset_index(drop=True, inplace=True)

In [128]:
print(df[df.overall >= 4].shape)
print(df[df.overall <= 2].shape)

(1500, 12)
(1500, 12)


In [129]:
# prepare
sentence_list, sentiment_label, sentence_senti_label, \
pos_neg_sentence_indices, pos_neg_sentiment_label, numSentence = prepare(df)

documents, sentence_list_again, bigram, documents_label\
= bigram_and_sentence(sentence_senti_label, sentence_list, numSentence, max_vocab=5000, threshold = 5, min_count = 20)

In [130]:
len(documents)

27123

In [120]:
def gensim_model(documents, window, dimension, epochs, save_path, save_per_epoch = 5, save=True):
    model = Doc2Vec(dm=1, dm_mean=1, negative =10,min_count=0, sample=1e-5,
                    window=window, size=dimension, 
                    workers=multiprocessing.cpu_count(), 
                    alpha=0.025, min_alpha=0.025)
    model.build_vocab(documents)
    decrease_rate = 0.025 / epochs
    for epoch in tqdm(range(epochs)):
        random.shuffle(documents)
        model.train(documents)
        model.alpha -= decrease_rate  # decrease the learning rate
        model.min_alpha = model.alpha  # fix the learning rate, no decay
        if (epoch + 1) % save_per_epoch ==0:
            if save:
                model.save(save_path + 'model_' + str(window) + '_' + str(dimension) + '_' + str(epoch+1))
    return model

In [135]:
# word / sentiment embedding

window = [3]
size = [100]
passes = 10
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, dm_mean=1, negative =10,min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=multiprocessing.cpu_count(), 
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
#             if (epoch + 1) % 5 ==0:
#                 model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:49<00:00, 11.39s/it]


In [4]:
model = Doc2Vec.load("E:/dataset/MasterThesis/gensim_models/test")

NotImplementedError: unknown URI scheme 'e' in 'E:/dataset/MasterThesis/gensim_models/test'

### test 1

In [148]:
model.save("E:/dataset/MasterThesis/gensim_models/test1")

In [149]:
wordVectors = np.zeros((len(model.index2word), model.vector_size))
for index, word in enumerate(model.index2word):
    wordVectors[index,:] = model[word]

sentimentVector = np.zeros((2, model.vector_size))
sentimentVector[0,:] = model.docvecs['positive']
sentimentVector[1,:] = model.docvecs['negative']

In [150]:
%%time
ratios = np.linspace(0,1,11)
for ratio in ratios:
    print("-----ratio is ", ratio)
    asum_embedding_path = "E:/dataset/MasterThesis/Models/ASUM_embedding" + str(ratio)
    asum_embedding = ASUM_Embedding(wordVectors, sentimentVector, numTopics=10, alpha=0.01, beta=0.001, gamma=1, binary=ratio, numSentiments=2)
    asum_embedding._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
    asum_embedding.run(sentence_list_again, save_path=asum_embedding_path, print_iter=10, save_iter = 50, maxIters= 100)

-----ratio is  0.0
Starting iteration 10 of 100
0.672
Starting iteration 20 of 100
0.601333333333
Starting iteration 30 of 100
0.6
Starting iteration 40 of 100
0.611666666667
Starting iteration 50 of 100
0.603666666667
Starting save model
Starting iteration 60 of 100
0.607
Starting iteration 70 of 100
0.610333333333
Starting iteration 80 of 100
0.612666666667
Starting iteration 90 of 100
0.614
Starting iteration 100 of 100
0.62
Starting save model
-----ratio is  0.1
Starting iteration 10 of 100
0.709666666667
Starting iteration 20 of 100
0.719
Starting iteration 30 of 100
0.734333333333
Starting iteration 40 of 100
0.735
Starting iteration 50 of 100
0.743666666667
Starting save model
Starting iteration 60 of 100
0.742666666667
Starting iteration 70 of 100
0.746
Starting iteration 80 of 100
0.735666666667
Starting iteration 90 of 100
0.738333333333
Starting iteration 100 of 100
0.732
Starting save model
-----ratio is  0.2
Starting iteration 10 of 100
0.731
Starting iteration 20 of 100
0

In [151]:
# word / sentiment embedding

window = [3]
size = [100]
passes = 5
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, dm_mean=1, negative =10,min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=multiprocessing.cpu_count(), 
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
#             if (epoch + 1) % 5 ==0:
#                 model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:42<00:00,  8.64s/it]


In [152]:
%%time

model.save("E:/dataset/MasterThesis/gensim_models/test2")
wordVectors = np.zeros((len(model.index2word), model.vector_size))
for index, word in enumerate(model.index2word):
    wordVectors[index,:] = model[word]

sentimentVector = np.zeros((2, model.vector_size))
sentimentVector[0,:] = model.docvecs['positive']
sentimentVector[1,:] = model.docvecs['negative']


ratios = np.linspace(0,1,11)
for ratio in ratios:
    print("-----ratio is ", ratio)
    asum_embedding_path = "E:/dataset/MasterThesis/Models/ASUM_embedding" + str(ratio)
    asum_embedding = ASUM_Embedding(wordVectors, sentimentVector, numTopics=10, alpha=0.01, beta=0.001, gamma=1, binary=ratio, numSentiments=2)
    asum_embedding._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
    asum_embedding.run(sentence_list_again, save_path=asum_embedding_path, print_iter=10, save_iter = 50, maxIters= 100)

-----ratio is  0.0
Starting iteration 10 of 100
0.671666666667
Starting iteration 20 of 100
0.676
Starting iteration 30 of 100
0.676666666667
Starting iteration 40 of 100
0.687666666667
Starting iteration 50 of 100
0.694
Starting save model
Starting iteration 60 of 100
0.701666666667
Starting iteration 70 of 100
0.703
Starting iteration 80 of 100
0.699666666667
Starting iteration 90 of 100
0.705
Starting iteration 100 of 100
0.709333333333
Starting save model
-----ratio is  0.1
Starting iteration 10 of 100
0.68
Starting iteration 20 of 100
0.673
Starting iteration 30 of 100
0.691333333333
Starting iteration 40 of 100
0.687333333333
Starting iteration 50 of 100
0.692666666667
Starting save model
Starting iteration 60 of 100
0.693666666667
Starting iteration 70 of 100
0.699333333333
Starting iteration 80 of 100
0.708
Starting iteration 90 of 100
0.706666666667
Starting iteration 100 of 100
0.717333333333
Starting save model
-----ratio is  0.2
Starting iteration 10 of 100
0.697333333333
S

In [153]:
# word / sentiment embedding

window = [5]
size = [100]
passes = 10
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, dm_mean=1, negative =10,min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=multiprocessing.cpu_count(), 
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
#             if (epoch + 1) % 5 ==0:
#                 model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:28<00:00,  8.95s/it]


In [154]:
%%time

model.save("E:/dataset/MasterThesis/gensim_models/test3")
wordVectors = np.zeros((len(model.index2word), model.vector_size))
for index, word in enumerate(model.index2word):
    wordVectors[index,:] = model[word]

sentimentVector = np.zeros((2, model.vector_size))
sentimentVector[0,:] = model.docvecs['positive']
sentimentVector[1,:] = model.docvecs['negative']


ratios = np.linspace(0,1,11)
for ratio in ratios:
    print("-----ratio is ", ratio)
    asum_embedding_path = "E:/dataset/MasterThesis/Models/ASUM_embedding" + str(ratio)
    asum_embedding = ASUM_Embedding(wordVectors, sentimentVector, numTopics=10, alpha=0.01, beta=0.001, gamma=1, binary=ratio, numSentiments=2)
    asum_embedding._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
    asum_embedding.run(sentence_list_again, save_path=asum_embedding_path, print_iter=10, save_iter = 50, maxIters= 100)

-----ratio is  0.0
Starting iteration 10 of 100
0.667
Starting iteration 20 of 100
0.649666666667
Starting iteration 30 of 100
0.636666666667
Starting iteration 40 of 100
0.632666666667
Starting iteration 50 of 100
0.641666666667
Starting save model
Starting iteration 60 of 100
0.633
Starting iteration 70 of 100
0.639333333333
Starting iteration 80 of 100
0.627666666667
Starting iteration 90 of 100
0.640666666667
Starting iteration 100 of 100
0.635666666667
Starting save model
-----ratio is  0.1
Starting iteration 10 of 100
0.672
Starting iteration 20 of 100
0.686
Starting iteration 30 of 100
0.699333333333
Starting iteration 40 of 100
0.701333333333
Starting iteration 50 of 100
0.709666666667
Starting save model
Starting iteration 60 of 100
0.696666666667
Starting iteration 70 of 100
0.708
Starting iteration 80 of 100
0.705
Starting iteration 90 of 100
0.697
Starting iteration 100 of 100
0.705333333333
Starting save model
-----ratio is  0.2
Starting iteration 10 of 100
0.689666666667


In [123]:
%%time
#parmeter search
model_path = "E:/dataset/MasterThesis/gensim_models/"
asum_embedding_path = "E:/dataset/MasterThesis/Models/ASUM_embedding"

window = [2,3,4,5]
size = [100,200,300]
num_topics = [2,5,10,20,50,100]
passes = 20
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, dm_mean=1, negative =10,min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=multiprocessing.cpu_count(), 
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
            if (epoch + 1) % 5 ==0:
                model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))
                
                # asum embedding model
                wordVectors = np.zeros((len(model.index2word), model.vector_size))
                for index, word in enumerate(model.index2word):
                    wordVectors[index,:] = model[word]

                sentimentVector = np.zeros((2, model.vector_size))
                sentimentVector[0,:] = model.docvecs['positive']
                sentimentVector[1,:] = model.docvecs['negative']
                for k in num_topics:
                    print("start!! num_of_topic : %s, size : %s, window : %s, epoch : %s"%(k,s,w,epoch+1))
                    start = time.time()
                    asum_embedding = ASUM_Embedding(wordVectors, sentimentVector, numTopics=k, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
                    asum_embedding._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
                    asum_embedding.run(sentence_list_again, save_path=asum_embedding_path, print_iter=10, save_iter = 20, maxIters= 100)
                    end = time.time()
                    print("END!! time :", start-end)

 20%|████████████████▌                                                                  | 4/20 [01:18<05:15, 19.69s/it]

start!! num_of_topic : 2, size : 100, window : 2, epoch : 5
Starting iteration 10 of 100
0.760142857143
Starting iteration 20 of 100
0.751428571429
Starting save model
Starting iteration 30 of 100
0.749285714286
Starting iteration 40 of 100
0.747428571429
Starting save model
Starting iteration 50 of 100
0.745285714286
Starting iteration 60 of 100
0.744571428571
Starting save model
Starting iteration 70 of 100
0.743428571429
Starting iteration 80 of 100
0.741571428571
Starting save model
Starting iteration 90 of 100
0.743571428571
Starting iteration 100 of 100
0.741285714286
Starting save model
END!! time : -1201.4870917797089
start!! num_of_topic : 5, size : 100, window : 2, epoch : 5
Starting iteration 10 of 100
0.755142857143
Starting iteration 20 of 100
0.748
Starting save model
Starting iteration 30 of 100
0.743571428571
Starting iteration 40 of 100
0.740285714286
Starting save model
Starting iteration 50 of 100
0.732714285714
Starting iteration 60 of 100
0.733142857143
Starting sa

KeyboardInterrupt: 

In [124]:
%%time
#parmeter search
model_path = "E:/dataset/MasterThesis/gensim_models/"
asum_embedding_path = "E:/dataset/MasterThesis/Models/ASUM_embedding"

window = [3,4,5]
size = [100,200,300]
num_topics = [50,30,20,10,5,2]
passes = 20
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, dm_mean=1, negative =10,min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=multiprocessing.cpu_count(), 
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
            if (epoch + 1) % 5 ==0:
                model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))
                
                # asum embedding model
                wordVectors = np.zeros((len(model.index2word), model.vector_size))
                for index, word in enumerate(model.index2word):
                    wordVectors[index,:] = model[word]

                sentimentVector = np.zeros((2, model.vector_size))
                sentimentVector[0,:] = model.docvecs['positive']
                sentimentVector[1,:] = model.docvecs['negative']
                for k in num_topics:
                    print("start!! num_of_topic : %s, size : %s, window : %s, epoch : %s"%(k,s,w,epoch+1))
                    start = time.time()
                    asum_embedding = ASUM_Embedding(wordVectors, sentimentVector, numTopics=k, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
                    asum_embedding._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
                    asum_embedding.run(sentence_list_again, save_path=asum_embedding_path, print_iter=10, save_iter = 20, maxIters= 100)
                    end = time.time()
                    print("END!! time :", end-start)

 20%|████████████████▌                                                                  | 4/20 [01:19<05:14, 19.68s/it]

start!! num_of_topic : 50, size : 100, window : 3, epoch : 5
Starting iteration 10 of 100
0.742428571429
Starting iteration 20 of 100
0.741857142857
Starting save model
Starting iteration 30 of 100
0.744142857143
Starting iteration 40 of 100
0.738857142857
Starting save model
Starting iteration 50 of 100
0.735
Starting iteration 60 of 100
0.728428571429
Starting save model
Starting iteration 70 of 100
0.725714285714
Starting iteration 80 of 100
0.721142857143
Starting save model
Starting iteration 90 of 100
0.720142857143
Starting iteration 100 of 100
0.715857142857
Starting save model
END!! time : -11032.702196121216
start!! num_of_topic : 30, size : 100, window : 3, epoch : 5


KeyboardInterrupt: 

In [125]:
asum_embedding.getTopKWordsByTopic(10)

,Topic1,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,...,Topic28,Topic29,Topic3,Topic30,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9
0,camera,camera,tv,camera,use,use,tv,use,camera,camera,...,one,batteri,camera,use,camera,use,use,use,use,camera
1,use,len,set,great,samsung,work,samsung,samsung,bought,one,...,samsung,camera,use,camera,one,return,time,samsung,one,canon
2,one,use,use,canon,one,one,one,one,one,use,...,appl,use,get,one,use,camera,get,problem,product,len
3,get,one,get,pictur,appl,time,use,tablet,use,purchas,...,get,tablet,tv,product,appl,one,hour,camera,would,would
4,tablet,get,would,love,get,veri,would,return,screen,product,...,drive,charg,samsung,would,product,amazon,tri,updat,appl,samsung
5,would,would,samsung,use,would,would,product,amazon,veri,get,...,use,video,one,like,get,work,batteri,issu,get,one
6,need,canon,pictur,bought,tv,tv,get,tv,great,would,...,would,gb,len,canon,buy,get,tablet,work,devic,get
7,video,veri,screen,len,amazon,like,work,camera,new,samsung,...,tv,turn,take,thing,purchas,buy,would,tablet,like,repair
8,work,tv,turn,one,like,app,purchas,product,love,tv,...,amazon,flash,pictur,veri,bought,love,one,one,tablet,use
9,also,time,sound,return,work,problem,set,would,ipad,new,...,new,ppi,would,work,canon,canon,work,app,comput,amazon


In [130]:
len(documents) / 5000

12.6234